# Core Tutorial

This script will introduce you to the basics of time series handling with pynapple.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pynapple as nap
import pandas as pd

## Time series object

Let's create a Tsd object with fake data. In this case, every time point is 1 second apart. A Tsd object is a wrapper of pandas series.

In [2]:
tsd = nap.Tsd(t = np.arange(100), d = np.random.rand(100), time_units = 's')

print(tsd)

Time (s)
0.0     0.926051
1.0     0.849987
2.0     0.034611
3.0     0.241067
4.0     0.931331
          ...   
95.0    0.388211
96.0    0.219992
97.0    0.327807
98.0    0.959152
99.0    0.416207
Length: 100, dtype: float64


It is possible to switch between seconds, milliseconds and microseconds. Note that when using *as_units*, the returned object is a simple pandas series.

In [3]:
print(tsd.as_units('ms'))
print(tsd.as_units('us'))

Time (ms)
0.0        0.926051
1000.0     0.849987
2000.0     0.034611
3000.0     0.241067
4000.0     0.931331
             ...   
95000.0    0.388211
96000.0    0.219992
97000.0    0.327807
98000.0    0.959152
99000.0    0.416207
Length: 100, dtype: float64
Time (us)
0           0.926051
1000000     0.849987
2000000     0.034611
3000000     0.241067
4000000     0.931331
              ...   
95000000    0.388211
96000000    0.219992
97000000    0.327807
98000000    0.959152
99000000    0.416207
Length: 100, dtype: float64


If only timestamps are available, for example spike times, we can construct a Ts object which holds only times. In this case, we generate 10 random spike times between 0 and 100 ms.

In [4]:
ts = nap.Ts(t = np.sort(np.random.uniform(0, 100, 10)), time_units = 'ms')

print(ts)

Time (s)
0.000332   NaN
0.007557   NaN
0.015512   NaN
0.017357   NaN
0.019130   NaN
0.036301   NaN
0.052718   NaN
0.068055   NaN
0.079949   NaN
0.099882   NaN
dtype: float64


If the time series contains multiple columns, we can use a TsdFrame.

In [5]:
tsdframe = nap.TsdFrame(t = np.arange(100), 
                        d = np.random.rand(100,3), 
                        time_units = 's', 
                        columns = ['a', 'b', 'c'])

print(tsdframe)

                 a         b         c
Time (s)                              
0.0       0.710777  0.045811  0.419415
1.0       0.010244  0.496336  0.253601
2.0       0.075987  0.079342  0.714916
3.0       0.195139  0.773036  0.518782
4.0       0.021775  0.313341  0.819941
...            ...       ...       ...
95.0      0.603578  0.319764  0.984949
96.0      0.788123  0.534558  0.886920
97.0      0.957802  0.675862  0.140417
98.0      0.393128  0.887271  0.061764
99.0      0.196731  0.357804  0.517005

[100 rows x 3 columns]


## Interval Sets object

The [IntervalSet](https://peyrachelab.github.io/pynapple/core.interval_set/) object stores multiple epochs with a common time units. It can then be used to restrict time series to this particular set of epochs.

In [6]:
epochs = nap.IntervalSet(start = [0, 10], end = [5, 15], time_units = 's')

new_tsd = tsd.restrict(epochs)

print(epochs)
print('\n')
print(new_tsd)

   start   end
0    0.0   5.0
1   10.0  15.0


Time (s)
0.0     0.926051
1.0     0.849987
2.0     0.034611
3.0     0.241067
4.0     0.931331
5.0     0.287283
10.0    0.603891
11.0    0.336611
12.0    0.631118
13.0    0.343460
14.0    0.608318
15.0    0.000967
dtype: float64


Multiple operations are available for IntervalSet. For example, IntervalSet can be merged. See the full documentation of the class [here](https://peyrachelab.github.io/pynapple/core.interval_set/#pynapple.core.interval_set.IntervalSet.intersect) for a list of all the functions that can be used to manipulate IntervalSets.

In [7]:
epoch1 = nap.IntervalSet(start=[0], end=[10]) # no time units passed. Default is us.
epoch2 = nap.IntervalSet(start=[5,30],end=[20,45])

epoch = epoch1.union(epoch2)
print(epoch1, '\n')
print(epoch2, '\n')
print(epoch)

   start   end
0    0.0  10.0 

   start   end
0    5.0  20.0
1   30.0  45.0 

   start   end
0    0.0  20.0
1   30.0  45.0


## TsGroup

Multiple time series with different time stamps (.i.e. a group of neurons with different spike times from one session) can be grouped with the TsGroup object. The TsGroup behaves like a dictionnary but it is also possible to slice with a list of indexes

In [8]:
my_ts = {0:nap.Ts(t = np.sort(np.random.uniform(0, 100, 1000)), time_units = 's'), # here a simple dictionnary
         1:nap.Ts(t = np.sort(np.random.uniform(0, 100, 2000)), time_units = 's'),
         2:nap.Ts(t = np.sort(np.random.uniform(0, 100, 3000)), time_units = 's')}

tsgroup = nap.TsGroup(my_ts)

print(tsgroup, '\n')
print(tsgroup[0], '\n') # dictionnary like indexing returns directly the Ts object
print(tsgroup[[0,2]]) # list like indexing

  Index    Freq. (Hz)
-------  ------------
      0         10
      1         20.01
      2         30.01 

Time (s)
0.019797    NaN
0.451827    NaN
0.464079    NaN
0.465697    NaN
0.517745    NaN
             ..
99.468552   NaN
99.492707   NaN
99.520110   NaN
99.849291   NaN
99.993950   NaN
Length: 1000, dtype: float64 

  Index    Freq. (Hz)
-------  ------------
      0         10
      2         30.01


Operations such as restrict can thus be directly applied to the TsGroup as well as other operations.

In [9]:
newtsgroup = tsgroup.restrict(epochs)

count = tsgroup.count(1, epochs, time_units='s') # Here counting the elements within bins of 1 seconds

print(count)

           0   1   2
Time (s)            
0.5        7  15  29
1.5       13  14  36
2.5        8  19  32
3.5        8  15  23
4.5       11  17  36
10.5      15  19  34
11.5      14  26  20
12.5      11  19  41
13.5      13  20  27
14.5       9  24  40


One advantage of grouping time series is that metainformation can be added about each elements. In this case, we add labels to each Ts object when instantiating the group and after. We can then use this label to split the group. See the documentation about [TsGroup](https://peyrachelab.github.io/pynapple/core.ts_group/) for all the ways to split TsGroup.

In [10]:
label1 = pd.Series(index=list(my_ts.keys()), data = [0,1,0])

tsgroup = nap.TsGroup(my_ts, time_units = 's', label1=label1)
tsgroup.set_info(label2=np.array(['a', 'a', 'b']))

print(tsgroup, '\n')

newtsgroup= tsgroup.getby_category('label1')
print(newtsgroup[0], '\n')
print(newtsgroup[1])


  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      0         10            0  a
      1         20.01         1  a
      2         30.01         0  b 

  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      0         10            0  a
      2         30.01         0  b 

  Index    Freq. (Hz)    label1  label2
-------  ------------  --------  --------
      1         20.01         1  a


## Time support

A key element of the manipulation of time series by pynapple is the inherent time support defined for Ts, Tsd, TsdFrame and TsGroup objects. The time support is defined as an IntervalSet that provides the time serie with a context. For example,, the restrict operation will update automatically the time support to the new time series. Ideally the time support should be defined for all time series when instantiating them. If no time series is given, the time support is inferred from the start and end of the time series. 

In this example, a TsGroup is instantiated with and without a time support. Notice how the frequency of each Ts element is changed when the time support is defined explicitely.

In [11]:
time_support = nap.IntervalSet(start = 0, end = 200, time_units = 's')

my_ts = {0:nap.Ts(t = np.sort(np.random.uniform(0, 100, 10)), time_units = 's'), # here a simple dictionnary
         1:nap.Ts(t = np.sort(np.random.uniform(0, 100, 20)), time_units = 's'),
         2:nap.Ts(t = np.sort(np.random.uniform(0, 100, 30)), time_units = 's')}

tsgroup = nap.TsGroup(my_ts)

tsgroup_with_time_support = nap.TsGroup(my_ts, time_support = time_support)

print(tsgroup, '\n')

print(tsgroup_with_time_support, '\n')

print(tsgroup_with_time_support.time_support) # acceding the time support

  Index    Freq. (Hz)
-------  ------------
      0          0.1
      1          0.2
      2          0.31 

  Index    Freq. (Hz)
-------  ------------
      0          0.05
      1          0.1
      2          0.15 

   start    end
0    0.0  200.0
